# Dataset Preprocessing Sweep
Trying to set up a new template for a notebook that will run a simple 5-fold cross-validation XGBoost model on a variety of dataset permutations.

# Setup

In [1]:
# two manual flags (ex-config)
COLAB = False
USE_GPU = True
libraries = ['xgboost', 'lightgbm', 'catboost', 'widedeep-SAINT']

In [2]:
# basic imports
from pathlib import Path
import os
import math
from datetime import datetime
import random

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
%matplotlib inline
%config Completer.use_jedi = False
os.environ['WANDB_NOTEBOOK_NAME'] = f"dataset_sweep_{datetime.now().strftime('%Y%m%d')}.ipynb"

In [5]:
# handle Google Colab-specific library installation/updating
if COLAB:
    # much of the below inspired by or cribbed from the May 2021 Kaggle Tabular Playground winner, at 
    # https://colab.research.google.com/gist/academicsuspect/0aac7bd6e506f5f70295bfc9a3dc2250/tabular-may-baseline.ipynb?authuser=1#scrollTo=LJoVKJb5wN0L
    
    # Kaggle API for downloading the datasets
#     !pip install --upgrade -q kaggle

    # weights and biases
    !pip install -qqqU wandb
    
    # Optuna for parameter search
    !pip install -q optuna

    # upgrade sklearn
    !pip install --upgrade scikit-learn

#     !pip install category_encoders
    
    if 'catboost' in libraries:
        !pip install catboost
    
    if 'xgboost' in libraries:
        if USE_GPU: 
            # this part is from https://github.com/rapidsai/gputreeshap/issues/24
            !pip install cmake --upgrade
            # !pip install sklearn --upgrade
            !git clone --recursive https://github.com/dmlc/xgboost
            %cd /content/xgboost
            !mkdir build
            %cd build
            !cmake .. -DUSE_CUDA=ON
            !make -j4
            %cd /content/xgboost/python-package
            !python setup.py install --use-cuda --use-nccl
            !/opt/bin/nvidia-smi
            !pip install shap
        else:
            !pip install --upgrade xgboost
    if 'lightgbm' in libraries:
        if USE_GPU:
            # lighgbm gpu compatible
            !git clone --recursive https://github.com/Microsoft/LightGBM
            ! cd LightGBM && rm -rf build && mkdir build && cd build && cmake -DUSE_GPU=1 ../../LightGBM && make -j4 && cd ../python-package && python3 setup.py install --precompile --gpu;
        else:
            !pip install --upgrade lightgbm
        

        

Now, non-stdlib imports

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

# general ML tooling
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from sklearn.metrics import log_loss, roc_auc_score
import wandb
from wandb.xgboost import wandb_callback
from wandb.lightgbm import wandb_callback
from sklearn.impute import SimpleImputer #, KNNImputer
import optuna
from optuna.integration.wandb import WeightsAndBiasesCallback
from optuna.samplers import TPESampler
from sklearn.utils import resample
import seaborn as sns

# from catboost import CatBoostClassifier
from xgboost import XGBClassifier
# from lightgbm import LGBMClassifier
# from sklearn.preprocessing import StandardScaler MinMaxScaler, MaxAbsScaler, RobustScaler, PolynomialFeatures
from sklearn.feature_selection import SelectKBest, f_regression
from joblib import dump, load
from sklearn.feature_selection import mutual_info_regression
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from boruta import BorutaPy
from BorutaShap import BorutaShap
import category_encoders as ce

In [28]:
if COLAB:
    # mount Google Drive
    from google.colab import drive
    drive.mount('/content/drive')
    
    # handling datapath
    datapath = Path('/content/drive/MyDrive/kaggle/tabular_playgrounds/oct2021/')
    
else:
    # if on local machine
#     datapath = Path('/media/sf/easystore/kaggle_data/tabular_playgrounds/sep2021/')  
    root = Path('/home/sf/code/kaggle/tabular_playgrounds/oct2021/')
    datapath = root/'datasets'
    edapath = root/'EDA'
    modelpath = Path('/media/sf/easystore/kaggle_data/tabular_playgrounds/oct2021/models/')
    predpath = root/'preds'
    subpath = root/'submissions'
    altdatapath = Path('/media/sf/easystore/kaggle_data/tabular_playgrounds/oct2021/alt_datasets/')
    studypath = root/'optuna_studies'
    
    for pth in [root, datapath, edapath, modelpath, predpath, subpath]:
        pth.mkdir(exist_ok=True)
    


In [8]:
SEED = 42

# Function to seed everything
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

seed_everything(seed=SEED)

## Ex-Model Config

In [9]:
# meta-config for preprocessing and cross-validation, but NOT for model parameters
exmodel_config = {
#     "feature_selector": SelectKBest,
#     "k_best": 80,
#     "feature_selection_scoring": f_regression,
#     'random_state': SEED,
#     'feature_generation': ['NaN_counts', 'SummaryStats', 'NaN_OneHots'],
#     'subsample': 1,
    'cross_val_strategy': KFold, # None for holdout, or the relevant sklearn class
    'kfolds': 5, # if 1, that means just doing holdout
    'test_size': 0.2,
#     'features_created': False,
#     'feature_creator': None,
}

## Data Loading

In [10]:
# loading the full training data set to get the feature correlations to target, for K-means clustering later
df = pd.read_feather(datapath/'train.feather')
# df_corr = df.corr() # getting the correlations of the features
# corr_target = df_corr.loc['target':'target'] # pulling out just the correlation of features with the target, as a 1-row df (for Series, it'd be df_corr.loc['target'])
corr_target = load(altdatapath/'corr_target.joblib')

In [11]:
corr_target.head()

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f276,f277,f278,f279,f280,f281,f282,f283,f284,target
target,0.004067,-0.029324,-0.015663,0.036279,0.019811,-0.012301,-0.012332,0.013528,-0.043557,-0.002662,...,-0.00329,-0.003869,-0.004477,-0.004503,-0.004319,-0.004587,-0.002426,-0.005901,-0.0037,1.0


In [12]:
corr_target_x = corr_target.drop('target', axis=1) # dropping the trivial 1.00 autocorrelation
corr_target_abs = abs(corr_target_x) # just interested in magnitudes here
corr_sorted = corr_target_abs.sort_values(by='target', axis=1, ascending=False) # df columns of useful values by correlation with target, will be modified later
y = df.target # pulling out the dependent variable
X = df.drop('target', axis=1) # isolating the independent variables
del df # cleaning up memory
categoricals = [f for f in X.columns if ((1000000 - X[f].nunique()) / 1000000) >=0.9 and X[f].nunique() > 2] # not touching already binary encoded vars

## Parameters

In [13]:
# optuna 20211004, thru 106 trials on unaltered original dataset
params = {
    'n_estimators': 3878,
    'max_depth': 4,
    'learning_rate': 0.024785857161974977,
    'reg_alpha': 26.867682044658245,
    'reg_lambda': 10.839759074147148,
    'subsample': 0.8208581489835881,
    'min_child_weight': 8.829122644339664,
    'colsample_bytree': 0.906420714280384,
    'gamma': 1.472322916021486
}

In [14]:
# b = load(altdatapath/'X_boruta_200iter_filtered_green.joblib')
# type(b)

In [15]:
# bdf = pd.DataFrame(b, index=X.index).join(y)
# bdf.head()

(Following cells generate the correlations for the different feature selections, so that the process need not be repeated each iteration.)

In [16]:
# b = load(altdatapath/'X_boruta_200iter_filtered_green.joblib')
# bdf = pd.DataFrame(b, index=X.index).join(y)
# bdf.head()
# bdf_corr = bdf.corr()
# bdf_corr_target = bdf_corr.loc['target':'target']
# bdf_corr_target_x = bdf_corr_target.drop('target', axis=1) # dropping the trivial 1.00 autocorrelation
# bdf_corr_target_abs = abs(bdf_corr_target_x) # just interested in magnitudes here
# bdf_corr_sorted = bdf_corr_target_abs.sort_values(by='target', axis=1, ascending=False)
# dump(bdf_corr_sorted, altdatapath/'X_boruta_200iter_filtered_green_corr_sorted.joblib')

In [17]:
# del b, bdf, bdf_corr_target, bdf_corr_target_x, bdf_corr_target_abs, bdf_corr_sorted

In [18]:
# b = pd.read_feather(altdatapath/'X_boruta_shap_200trials.feather')
# bdf = b.join(y)
# # bdf.head()
# bdf_corr = bdf.corr()
# bdf_corr_target = bdf_corr.loc['target':'target']
# bdf_corr_target_x = bdf_corr_target.drop('target', axis=1) # dropping the trivial 1.00 autocorrelation
# bdf_corr_target_abs = abs(bdf_corr_target_x) # just interested in magnitudes here
# bdf_corr_sorted = bdf_corr_target_abs.sort_values(by='target', axis=1, ascending=False)
# dump(bdf_corr_sorted, altdatapath/'X_boruta_shap_200trials_corr_sorted.joblib')

In [30]:
# originally from https://www.kaggle.com/satorushibata/optimize-catboost-hyperparameter-with-optuna-gpu
def objective(trial, X=X, y=y, categoricals=categoricals, corr_sorted=corr_sorted):
    # split the (original Kaggle training) data into partitions
    # if study.best_trial:
    #     print("Dumping best params, which are:")
    #     print(str(study.best_trial.params))
    #     dump(study.best_trial.params, filename=datapath/'optuna_catboost_best_20210920.joblib')
    

#     dump(pca60, edapath/'PCA_60.joblibg')

    # use the original 286-feature dataset, or the 136-feature BorutaShap selected one
#     dataset = trial.suggest_categorical('dataset', ['X_orig.feather', 'X_boruta_shap_200trials.feather']) 
#     train_source = altdatapath/'X_orig.feather'
    # train_source = altdatapath/'train-WITH-KMeans_12cluster_kmeans++_maxiter1000_rs42.feather' #'X_boruta_shap_200trials.feather'
#     X = pd.read_feather(path=train_source)
#     y = load(datapath/'y.joblib')
    
#     # decides whether binary-encoded categoricals are encoded or not
#     cardinality_min = trial.suggest_categorical('cardinality_min', [0, 2]) 
        
    encoder_name = trial.suggest_categorical('encoder_name', ['woe', 'catboost', 'james-stein', 'loo', 'mestimate', 'target', 'hashing', None])
    if encoder_name:
        encode_before_kmeans = trial.suggest_categorical('encode_before_kmeans', [True, False]) # determines order
    
    # feature selection setup -- applied before preprocessing
    feature_selection = trial.suggest_categorical('feature_selection', ['BorutaShap', 'Boruta', None])
    k_means_method = trial.suggest_categorical('k_means_method', [25, 50, 100, 'k-means++', None]) # K-Means initialization method

    # now, switch datasets if feature selection is implemented; regardless, prepare appropriate K-Means setup (to be implemented later, in folds)
    if feature_selection: # create a subset of features if appropriate
        if feature_selection == 'BorutaShap':
            X = pd.read_feather(altdatapath/'X_boruta_shap_200trials.feather') # :: pd.DataFrame
            categoricals = [f for f in X.columns if ((1000000 - X[f].nunique()) / 1000000) >=0.9 and X[f].nunique() > 2] # not touching already binary encoded vars
            # k-means cluster feature generation setup 
            if k_means_method:
                corr_sorted = load(altdatapath/'X_boruta_shap_200trials_corr_sorted.joblib') # load prepared correlations
                k_means_clusters = trial.suggest_int('k_means_clusters', 6, 12) # for grabbing the most useful features from `corr_sorted`
                useful_features = list(corr_sorted.columns[:k_means_clusters])
        elif feature_selection == 'Boruta':
            X = pd.DataFrame(load(altdatapath/'X_boruta_200iter_filtered_green.joblib'), index=X.index)
            if k_means_method:
                corr_sorted = load(altdatapath/'X_boruta_200iter_filtered_green_corr_sorted.joblib') # load prepared correlations
                k_means_clusters = trial.suggest_int('k_means_clusters', 6, 12) # for grabbing the most useful features from `corr_sorted`
                useful_features = list(corr_sorted.columns[:k_means_clusters])
            categoricals = [f for f in X.columns if ((1000000 - X[f].nunique()) / 1000000) >=0.9 and X[f].nunique() > 2] # not touching already binary encoded vars
    else:
        if k_means_method:
            k_means_clusters = trial.suggest_int('k_means_clusters', 6, 12) # for grabbing the most useful features from `corr_sorted`
            useful_features = list(corr_sorted.columns[:k_means_clusters])
    
    # define dict of encoders, with names as keys and implementations as values
    encoders = {
        'woe': ce.WOEEncoder(cols=categoricals),
        'catboost': ce.CatBoostEncoder(cols=categoricals),
        'james-stein': ce.JamesSteinEncoder(cols=categoricals),
        'loo': ce.LeaveOneOutEncoder(cols=categoricals),
        'mestimate': ce.MEstimateEncoder(cols=categoricals),
        'target': ce.TargetEncoder(cols=categoricals),
        'hashing': ce.HashingEncoder(cols=categoricals),
    }
    
    # PCA dimensionality reduction setup -- applied at end of preprocessing
    pca_components = trial.suggest_categorical('pca_components', [50, 75, 'mle', None, 'NO'])
    
    # define k-fold splitter
    kfold = KFold(n_splits=5, shuffle=False)
        
    # initialize lists for out-of-fold preds and ground truth
    oof_preds, oof_y = [], []
            
    for fold, (train_ids, valid_ids) in enumerate(kfold.split(X,y)):
        print(f"FOLD {fold}")
        print("---------------------------------------------------")
        
        y_train, y_valid = y[train_ids], y[valid_ids] # slicing syntax works on both pandas.Series and numpy.ndarray
        # category_encoders expects pandas.DataFrames
        X_train, X_valid = X.iloc[train_ids,:], X.iloc[valid_ids,:] # bc need pandas.DataFrames for slicing
        
        # now, apply preprocessing
        if encoder_name: # if categorical encoding to be applied to high cardinality (2<x<100,000) categoricals...
            if k_means_method: # if k-means proceeding
                if encode_before_kmeans: # do category encoding, then clustering
                    # category encoding for high-cardinality categoricals
                    encoder = encoders[encoder_name]
                    X_train = encoder.fit_transform(X_train, y_train)
                    X_valid = encoder.transform(X_valid)

                    # k-means cluster feature generation
                    cluster_cols = [f"cluster{i+1}" for i in range(k_means_clusters)]
                    if k_means_method == 'k-means++':
                        kmeans = KMeans(n_clusters=k_means_clusters, init="k-means++", max_iter=1000, random_state=SEED,n_jobs=-1)
                    else:
                        kmeans = KMeans(n_clusters=k_means_clusters, n_init=k_means_method, max_iter=1000, random_state=SEED, n_jobs=-1)
                    # fit on the training set only
                    X_train_clusters = kmeans.fit_transform(X_train[useful_features])
                    X_valid_clusters = kmeans.transform(X_valid[useful_features])
                    # convert numpy.ndarrays back to properly-labeled pandas.DataFrames
                    X_train_clusters = pd.DataFrame(X_train_clusters, columns=cluster_cols, index=X_train.index)
                    X_valid_clusters = pd.DataFrame(X_valid_clusters, columns=cluster_cols, index=X_valid.index)
                    # join the cluster-distance features to the training and validation sets
                    X_train = X_train.join(X_train_clusters)
                    X_valid = X_valid.join(X_valid_clusters)

                else: # do k-means clustering, then do category encoding
                    cluster_cols = [f"cluster{i+1}" for i in range(k_means_clusters)]
                    if k_means_method == 'k-means++':
                        kmeans = KMeans(n_clusters=k_means_clusters, init="k-means++", max_iter=1000, random_state=SEED,n_jobs=-1)
                    else:
                        kmeans = KMeans(n_clusters=k_means_clusters, n_init=k_means_method, max_iter=1000, random_state=SEED, n_jobs=-1)
                    X_train_clusters = kmeans.fit_transform(X_train[useful_features])
                    X_valid_clusters = kmeans.transform(X_valid[useful_features])
                    X_train_clusters = pd.DataFrame(X_train_clusters, columns=cluster_cols, index=X_train.index)
                    X_valid_clusters = pd.DataFrame(X_valid_clusters, columns=cluster_cols, index=X_valid.index)
                    X_train = X_train.join(X_train_clusters)
                    X_valid = X_valid.join(X_valid_clusters)

                    encoder = encoders[encoder_name]
                    X_train = encoder.fit_transform(X_train, y_train)
                    X_valid = encoder.transform(X_valid)
            
            else: # category encoding, but no k-means
                encoder = encoders[encoder_name]
                X_train = encoder.fit_transform(X_train, y_train)
                X_valid = encoder.transform(X_valid)
                
        else: # no category encoding
            if k_means_method: # if still doing k-means
                # k-means cluster feature generation
                cluster_cols = [f"cluster{i+1}" for i in range(k_means_clusters)]
                if k_means_method == 'k-means++':
                    kmeans = KMeans(n_clusters=k_means_clusters, init="k-means++", max_iter=1000, random_state=SEED,n_jobs=-1)
                else:
                    kmeans = KMeans(n_clusters=k_means_clusters, n_init=k_means_method, max_iter=1000, random_state=SEED, n_jobs=-1)
                # fit on the training set only
                X_train_clusters = kmeans.fit_transform(X_train[useful_features])
                X_valid_clusters = kmeans.transform(X_valid[useful_features])
                # convert numpy.ndarrays back to properly-labeled pandas.DataFrames
                X_train_clusters = pd.DataFrame(X_train_clusters, columns=cluster_cols, index=X_train.index)
                X_valid_clusters = pd.DataFrame(X_valid_clusters, columns=cluster_cols, index=X_valid.index)
                # join the cluster-distance features to the training and validation sets
                X_train = X_train.join(X_train_clusters)
                X_valid = X_valid.join(X_valid_clusters)
            
        
        # now, PCA dimensionality reduction
        if pca_components != 'NO':
            pca = PCA(n_components=pca_components, random_state=42)
            X_train = pca.fit_transform(X_train)
            X_valid = pca.transform(X_valid)
            
        # define models
        model = XGBClassifier(
            booster='gbtree',
            tree_method='gpu_hist',
            random_state=42,
            n_jobs=-1, 
            verbosity=1, 
            objective='binary:logistic',
            **params)
        model.fit(X_train, y_train)
        y_valid_preds = model.predict_proba(X_valid)[:,1]

        # add the fold-model's OOF preds and ground truths to the out-of-loop lists
        oof_preds.extend(y_valid_preds)
        oof_y.extend(y_valid)


        fold_valid_auc = roc_auc_score(y_valid, y_valid_preds)
        print(f"Valid AUC for fold {fold} is {fold_valid_auc}")   
#         dump(model, Path(runpath/f"{library}_fold{fold}_rs{random_state}_model.joblib"))

    model_valid_auc = roc_auc_score(oof_y, oof_preds)
    print(f"Valid AUC score for is {model_valid_auc}")
    
    return model_valid_auc

In [20]:
# meta-config for preprocessing and cross-validation, but NOT for model parameters
# in the sweep version, this includes both ex-model parameters and defaults for model parameters
exmodel_config = {
    # model config
    "library": 'xgboost',
#     "model": XGBClassifier,
#     "n_estimators": 100, 
#     "max_depth": 3,
#     "learning_rate": 0.1,
#     "test_size": 0.2,
#     "reg_lambda": None, 
#     "scaler": "sklearn.preprocessing.StandardScaler()", # TODO: experiment with others (but imputation may be slow)
#     "scale_b4_impute": False,
#     "imputer": "sklearn.impute.SimpleImputer(strategy='median', add_indicator=True)",
#     "knn_imputer_n_neighbors": None, # None if a different imputer is used
#     "feature_selector": SelectKBest,
#     "k_best": 80,
#     "feature_selection_scoring": f_regression,
    'random_state': SEED,
    'optuna': True,
#     'optuna_trials': 50,
#     'subsample': 1,
#     'cross_val_strategy': None, # None for holdout, or the relevant sklearn class
#     'kfolds': 1, # if 1, that means just doing holdout
#     'test_size': 0.2,
    # these are XGBoost default (my choice) params 
#     "tree_method": "auto", # set to 'gpu_hist' to try GPU if available
#     "booster": 'gbtree', # dart may be marginally better, but will opt for this quicker approach as a default
#     "n_estimators": 200, 
#     "max_depth": 3,
#     "learning_rate": 0.1,
#     "n_jobs": -1,
#     "verbosity": 1,
#     "subsample": 1,
#     'features_created': False,
#     'feature_creator': None,
}

wandb_kwargs = {
    # wandb config
    'name': f"{os.environ['WANDB_NOTEBOOK_NAME'][:-6]}_{datetime.now().strftime('%H%M%S')}", # just removes the .ipynb extension, leaving the notebook filename's stem
    'project': '202110_Kaggle_tabular_playground',
    'tags': ['sweep'],
    'notes': "Sweep for preprocessing techniques on dataset",
    'config': exmodel_config,
}

In [21]:
wandbc = WeightsAndBiasesCallback(wandb_kwargs=wandb_kwargs)

wandb: WARNING WANDB_NOTEBOOK_NAME should be a path to a notebook file, couldn't find dataset_sweep_20211026.ipynb
wandb: Currently logged in as: hushifang (use `wandb login --relogin` to force relogin)


In [22]:
# study = optuna.create_study(direction = "maximize", sampler = TPESampler(seed=int(SEED)), study_name='dataset_20211026')
study = load()

[I 2021-10-26 12:44:50,241] A new study created in memory with name: dataset_20211026


In [31]:
for x in range(11,100):
    study.optimize(objective, n_trials = 1, callbacks = [wandbc]) #n_jobs = multiprocessing.cpu_count())
    dump(study, filename=studypath/f'optuna_dataset_study_trial{x}_20211026.joblib')

FOLD 0
---------------------------------------------------
[16:22:58] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Valid AUC for fold 0 is 0.7517760922401533
FOLD 1
---------------------------------------------------
[16:28:29] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Valid AUC for fold 1 is 0.7539213843954606
FOLD 2
---------------------------------------------------
[16:34:08] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore

[I 2021-10-26 16:48:30,187] Trial 12 finished with value: 0.7542908472902581 and parameters: {'encoder_name': 'target', 'encode_before_kmeans': False, 'feature_selection': None, 'k_means_method': None, 'pca_components': None}. Best is trial 6 with value: 0.8529520938646249.


Valid AUC score for is 0.7542908472902581
FOLD 0
---------------------------------------------------
[16:49:01] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Valid AUC for fold 0 is 0.8460498790948396
FOLD 1
---------------------------------------------------
[16:53:21] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Valid AUC for fold 1 is 0.8487617952918354
FOLD 2
---------------------------------------------------
[16:57:40] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitl

[I 2021-10-26 17:10:11,751] Trial 13 finished with value: 0.846609851097209 and parameters: {'encoder_name': 'loo', 'encode_before_kmeans': False, 'feature_selection': None, 'k_means_method': None, 'pca_components': None}. Best is trial 6 with value: 0.8529520938646249.


Valid AUC score for is 0.846609851097209
FOLD 0
---------------------------------------------------
[17:10:37] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Valid AUC for fold 0 is 0.8468150822382937
FOLD 1
---------------------------------------------------
[17:14:52] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Valid AUC for fold 1 is 0.8491576487090565
FOLD 2
---------------------------------------------------
[17:19:07] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly

...

[I 2021-10-27 05:50:04,653] Trial 49 finished with value: 0.847097402563946 and parameters: {'encoder_name': None, 'feature_selection': None, 'k_means_method': None, 'pca_components': None}. Best is trial 6 with value: 0.8529520938646249.


Valid AUC score for is 0.847097402563946
FOLD 0
---------------------------------------------------
[05:50:22] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Valid AUC for fold 0 is 0.8467395508145958
FOLD 1
---------------------------------------------------
[05:54:30] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Valid AUC for fold 1 is 0.8490648949947982
FOLD 2
---------------------------------------------------
[05:58:38] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly

[I 2021-10-27 06:10:48,434] Trial 50 finished with value: 0.847097402563946 and parameters: {'encoder_name': None, 'feature_selection': None, 'k_means_method': None, 'pca_components': None}. Best is trial 6 with value: 0.8529520938646249.


Valid AUC score for is 0.847097402563946
FOLD 0
---------------------------------------------------
[06:11:16] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Valid AUC for fold 0 is 0.8460498790948396
FOLD 1
---------------------------------------------------
[06:15:32] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Valid AUC for fold 1 is 0.8487617952918354
FOLD 2
---------------------------------------------------
[06:19:49] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly

KeyboardInterrupt: 

In [32]:
study.best_trial.params

{'encoder_name': 'loo',
 'encode_before_kmeans': True,
 'feature_selection': 'Boruta',
 'k_means_method': 100,
 'k_means_clusters': 7,
 'pca_components': 'NO'}

In [33]:
wandb.log({'best_dataset_params': study.best_trial.params})
wandb.finish()

encode_before_kmeans,▁████████▁▁▁▁▁
k_means_clusters,▄▂▁▁█▄█
k_means_method,█▃██▁▃
pca_components,██▁█
value,█▃▂▆█▄▇▇▁███████████████████████████████
encode_before_kmeans,False
k_means_clusters,12
value,0.8471


Now, the baseline for comparison:

In [35]:
# define k-fold splitter
kfold = KFold(n_splits=5, shuffle=False)

# initialize lists for out-of-fold preds and ground truth
oof_preds, oof_y = [], []

for fold, (train_ids, valid_ids) in enumerate(kfold.split(X,y)):
    print(f"FOLD {fold}")
    print("---------------------------------------------------")

    y_train, y_valid = y[train_ids], y[valid_ids] # slicing syntax works on both pandas.Series and numpy.ndarray
    # category_encoders expects pandas.DataFrames
    X_train, X_valid = X.iloc[train_ids,:], X.iloc[valid_ids,:] # bc need pandas.DataFrames for slicing


    # define models
    model = XGBClassifier(
        booster='gbtree',
        tree_method='gpu_hist',
        random_state=42,
        n_jobs=-1, 
        verbosity=1, 
        objective='binary:logistic',
        **params)
    model.fit(X_train, y_train)
    y_valid_preds = model.predict_proba(X_valid)[:,1]

    # add the fold-model's OOF preds and ground truths to the out-of-loop lists
    oof_preds.extend(y_valid_preds)
    oof_y.extend(y_valid)


    fold_valid_auc = roc_auc_score(y_valid, y_valid_preds)
    print(f"Valid AUC for fold {fold} is {fold_valid_auc}")   
#         dump(model, Path(runpath/f"{library}_fold{fold}_rs{random_state}_model.joblib"))

model_valid_auc = roc_auc_score(oof_y, oof_preds)
print(f"Valid AUC score for is {model_valid_auc}")

FOLD 0
---------------------------------------------------
[07:59:57] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Valid AUC for fold 0 is 0.8562083823462339
FOLD 1
---------------------------------------------------
[08:06:01] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Valid AUC for fold 1 is 0.8583507447326876
FOLD 2
---------------------------------------------------
[08:12:06] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore

So, the best of the sweep was trial 6 with AUC of 0.8529520938646249, but the straight-up analysis without any bells and whistles gets 0.8566651115202035. 

Conclusion: best to forget about preprocessing. $\blacksquare$